In [ ]:
# import 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
df = pd.read_csv('Data/train.csv',na_values=None)
df.head()

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(df.corr(),annot=True,cmap='BrBG')
plt.show()

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1,test_size=0.2)
for train_indices, test_indices in split.split(df,df[['Survived','Pclass','Sex']]):
    start_train_set = df.loc[train_indices]
    start_test_set = df.loc[test_indices]

In [ ]:
from sklearn.base import (
    BaseEstimator,
    TransformerMixin
)
from sklearn.impute import(
    SimpleImputer
)


class AgeImputer(BaseEstimator,TransformerMixin):
    
    def fit(self,X,y=None):
        return self
    
    def transform(self,X):
        imputer = SimpleImputer(strategy='mean')
        X["Age"]=imputer.fit_transform(X[['Age']])
        return X

In [ ]:
# One Hot Coding
from sklearn.preprocessing import OneHotEncoder

class FeaturEncoder(BaseEstimator,TransformerMixin):
    
    def fit(self, X,y=None):
        return self
    
    def transform(self,X):
        encoder = OneHotEncoder()
        matrix = encoder.fit_transform(X[['Embarked']]).toarray()
        
        column_names =['C','S','Q',"N"]
        
        for i in range(len(matrix.T)):
            X[column_names[i]]=matrix[i]
        matrix = encoder.fit_transform(X[['Sex']]).toarray()
        column_names = ['Female','Male']
        for i in range(len(matrix.T)):
            X[column_names[i]]=matrix[i]
        return X

In [ ]:
class FeaturDroper(BaseEstimator,TransformerMixin):
    def fit(self,X,y=None):
        return self
    def transform(self,X):
        return X.drop(['Embarked','Name','Ticket','Cabin','Sex',"N"],axis=1,errors = "ignore")

In [ ]:
#pipLine
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('ageImputer',AgeImputer()),
                    ('featurencoder',FeaturEncoder()),
                    ('featurDroper',FeaturDroper())])

In [ ]:
start_train_set = pipeline.fit_transform(start_train_set)

In [ ]:
df.head()